# 📸 Data Collection - Sign Language Gestures

This notebook helps you collect hand landmark data for sign language gestures using your webcam and MediaPipe.

## Objectives
- Define sign language classes to collect
- Use MediaPipe to detect and extract hand landmarks
- Collect multiple samples per gesture class
- Save data for preprocessing

## What You'll Collect
- **21 hand landmarks** per hand (x, y, z coordinates)
- **63 features** total (21 landmarks × 3 coordinates)
- **Multiple samples** per gesture for robust training

---

## 1. Import Required Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## 2. Initialize MediaPipe

In [ ]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("✅ MediaPipe initialized")
print(f"   MediaPipe version: {mp.__version__}")

## 3. Configuration Settings

In [ ]:
# Data collection settings
DATA_DIR = 'data/raw'
os.makedirs(DATA_DIR, exist_ok=True)

# Define your sign language classes
# Start with ASL alphabet letters or common words
CLASSES = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L']

# Number of samples to collect per class
SAMPLES_PER_CLASS = 200  # Increase for better accuracy

# MediaPipe detection settings
MIN_DETECTION_CONFIDENCE = 0.7
MIN_TRACKING_CONFIDENCE = 0.7

print("\n" + "="*60)
print("DATA COLLECTION CONFIGURATION")
print("="*60)
print(f"Classes to collect: {CLASSES}")
print(f"Total classes: {len(CLASSES)}")
print(f"Samples per class: {SAMPLES_PER_CLASS}")
print(f"Total samples to collect: {len(CLASSES) * SAMPLES_PER_CLASS}")
print(f"Detection confidence: {MIN_DETECTION_CONFIDENCE}")
print(f"Tracking confidence: {MIN_TRACKING_CONFIDENCE}")
print("="*60)

## 4. Data Collection Function

In [ ]:
def collect_data_for_class(class_name, num_samples=200):
    """
    Collect hand landmark data for a specific sign language gesture.
    
    Args:
        class_name (str): Name of the gesture class (e.g., 'A', 'B', 'Hello')
        num_samples (int): Number of samples to collect
    
    Returns:
        tuple: (data, labels) - Lists of landmark data and corresponding labels
    """
    data = []
    labels = []
    
    cap = cv2.VideoCapture(0)
    
    # Set camera properties for better quality
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,  # Collect one hand at a time
        min_detection_confidence=MIN_DETECTION_CONFIDENCE,
        min_tracking_confidence=MIN_TRACKING_CONFIDENCE
    ) as hands:
        
        print(f"\n{'='*60}")
        print(f"Collecting data for: {class_name}")
        print(f"{'='*60}")
        print("\nInstructions:")
        print("  1. Position your hand in front of the camera")
        print("  2. Make the sign for '{}'" .format(class_name))
        print("  3. Press SPACE to start collecting")
        print("  4. Keep your hand steady and visible")
        print("  5. Press ESC to skip this class\n")
        
        # Wait for user to get ready
        ready = False
        while not ready:
            ret, frame = cap.read()
            if not ret:
                print("❌ Error: Cannot read from webcam")
                break
                
            frame = cv2.flip(frame, 1)  # Mirror the image
            
            # Add instructions overlay
            cv2.rectangle(frame, (0, 0), (frame.shape[1], 120), (0, 0, 0), -1)
            cv2.putText(frame, f"Class: {class_name}", (20, 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            cv2.putText(frame, "Press SPACE to start | ESC to skip", (20, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            cv2.imshow('Data Collection', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == 32:  # SPACE
                ready = True
            elif key == 27:  # ESC
                print(f"⏭️  Skipped class: {class_name}")
                cap.release()
                cv2.destroyAllWindows()
                return data, labels
        
        # Countdown before collection
        for i in range(3, 0, -1):
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)
            cv2.putText(frame, f"Starting in {i}...", (frame.shape[1]//2 - 150, frame.shape[0]//2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 4)
            cv2.imshow('Data Collection', frame)
            cv2.waitKey(1000)
        
        # Collect samples
        sample_count = 0
        failed_detections = 0
        
        print(f"\n📸 Collecting {num_samples} samples...")
        
        with tqdm(total=num_samples, desc=f"Class {class_name}") as pbar:
            while sample_count < num_samples:
                ret, frame = cap.read()
                if not ret:
                    break
                
                frame = cv2.flip(frame, 1)
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process with MediaPipe
                results = hands.process(rgb_frame)
                
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        # Draw landmarks on frame
                        mp_drawing.draw_landmarks(
                            frame,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style()
                        )
                        
                        # Extract landmarks (x, y, z for each of 21 landmarks)
                        landmarks = []
                        for lm in hand_landmarks.landmark:
                            landmarks.extend([lm.x, lm.y, lm.z])
                        
                        # Store data
                        data.append(landmarks)
                        labels.append(class_name)
                        sample_count += 1
                        pbar.update(1)
                        
                        # Visual feedback - green border when capturing
                        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), 
                                    (0, 255, 0), 10)
                else:
                    failed_detections += 1
                    # Red border when no hand detected
                    cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), 
                                (0, 0, 255), 10)
                
                # Display progress overlay
                cv2.rectangle(frame, (0, 0), (frame.shape[1], 120), (0, 0, 0), -1)
                cv2.putText(frame, f"Class: {class_name}", (20, 40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                cv2.putText(frame, f"Progress: {sample_count}/{num_samples}", (20, 80), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                
                # Progress bar
                bar_width = int((sample_count / num_samples) * (frame.shape[1] - 40))
                cv2.rectangle(frame, (20, 95), (20 + bar_width, 110), (0, 255, 0), -1)
                cv2.rectangle(frame, (20, 95), (frame.shape[1] - 20, 110), (255, 255, 255), 2)
                
                cv2.imshow('Data Collection', frame)
                
                if cv2.waitKey(1) & 0xFF == 27:  # ESC to stop
                    print(f"\n⏹️  Collection stopped by user")
                    break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Summary
    print(f"\n✅ Collection complete for '{class_name}'")
    print(f"   Samples collected: {len(data)}")
    print(f"   Failed detections: {failed_detections}")
    if len(data) > 0:
        success_rate = (len(data) / (len(data) + failed_detections)) * 100
        print(f"   Success rate: {success_rate:.1f}%")
    
    return data, labels

## 5. Collect Data for All Classes

In [ ]:
# Initialize storage
all_data = []
all_labels = []

print("\n" + "="*60)
print("STARTING DATA COLLECTION")
print("="*60)
print(f"Total classes: {len(CLASSES)}")
print(f"Samples per class: {SAMPLES_PER_CLASS}")
print(f"Estimated time: ~{len(CLASSES) * 2} minutes\n")

# Collect data for each class
for idx, class_name in enumerate(CLASSES, 1):
    print(f"\n[{idx}/{len(CLASSES)}] Preparing to collect: {class_name}")
    
    data, labels = collect_data_for_class(class_name, SAMPLES_PER_CLASS)
    
    all_data.extend(data)
    all_labels.extend(labels)
    
    # Short break between classes
    if idx < len(CLASSES):
        print(f"\n⏸️  Take a 5-second break before next class...")
        import time
        time.sleep(5)

print("\n" + "="*60)
print("DATA COLLECTION COMPLETE!")
print("="*60)
print(f"Total samples collected: {len(all_data)}")
print(f"Total labels: {len(all_labels)}")
print(f"Unique classes: {len(set(all_labels))}")

## 6. Visualize Collected Data

In [ ]:
# Count samples per class
from collections import Counter

class_counts = Counter(all_labels)

# Create visualization
plt.figure(figsize=(12, 6))
classes = list(class_counts.keys())
counts = list(class_counts.values())

bars = plt.bar(classes, counts, color='skyblue', edgecolor='navy', linewidth=1.5)

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.xlabel('Sign Language Class', fontsize=12, fontweight='bold')
plt.ylabel('Number of Samples', fontsize=12, fontweight='bold')
plt.title('Collected Data Distribution', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()

# Save plot
os.makedirs('outputs/visualizations', exist_ok=True)
plt.savefig('outputs/visualizations/data_collection_distribution.png', 
            dpi=300, bbox_inches='tight')
plt.show()

# Print statistics
print("\nClass Distribution:")
print("="*40)
for class_name, count in sorted(class_counts.items()):
    print(f"  {class_name:10s}: {count:4d} samples")
print("="*40)
print(f"  {'Total':10s}: {sum(counts):4d} samples")

## 7. Save Collected Data

In [ ]:
# Convert to numpy arrays
data_array = np.array(all_data)
labels_array = np.array(all_labels)

print("Data shape:", data_array.shape)
print("Labels shape:", labels_array.shape)

# Save as numpy files
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
data_filename = os.path.join(DATA_DIR, f'landmarks_{timestamp}.npy')
labels_filename = os.path.join(DATA_DIR, f'labels_{timestamp}.npy')

np.save(data_filename, data_array)
np.save(labels_filename, labels_array)

# Also save as latest (for easy loading)
np.save(os.path.join(DATA_DIR, 'landmarks.npy'), data_array)
np.save(os.path.join(DATA_DIR, 'labels.npy'), labels_array)

print(f"\n✅ Data saved successfully!")
print(f"   Timestamped files:")
print(f"     - {data_filename}")
print(f"     - {labels_filename}")
print(f"   Latest files:")
print(f"     - data/raw/landmarks.npy")
print(f"     - data/raw/labels.npy")

## 8. Create Metadata File

In [ ]:
# Create metadata dictionary
metadata = {
    'collection_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'total_samples': len(all_data),
    'num_classes': len(set(all_labels)),
    'classes': sorted(list(set(all_labels))),
    'samples_per_class': dict(class_counts),
    'feature_dimension': len(all_data[0]) if all_data else 0,
    'mediapipe_version': mp.__version__,
    'detection_confidence': MIN_DETECTION_CONFIDENCE,
    'tracking_confidence': MIN_TRACKING_CONFIDENCE
}

# Save metadata as JSON
import json

metadata_filename = os.path.join(DATA_DIR, f'metadata_{timestamp}.json')
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=4)

# Also save as latest
with open(os.path.join(DATA_DIR, 'metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=4)

print("\n✅ Metadata saved!")
print("\nMetadata Summary:")
print(json.dumps(metadata, indent=2))

## 9. Data Quality Check

In [ ]:
# Check for any issues
print("\n" + "="*60)
print("DATA QUALITY CHECK")
print("="*60)

# Check for missing values
has_nan = np.isnan(data_array).any()
print(f"\n1. Missing values (NaN): {'❌ Found' if has_nan else '✅ None'}")

# Check for infinite values
has_inf = np.isinf(data_array).any()
print(f"2. Infinite values: {'❌ Found' if has_inf else '✅ None'}")

# Check data shape consistency
expected_features = 63  # 21 landmarks × 3 coordinates
actual_features = data_array.shape[1]
print(f"3. Feature dimension: {actual_features} {'✅ Correct' if actual_features == expected_features else '❌ Incorrect'}")

# Check class balance
min_samples = min(class_counts.values())
max_samples = max(class_counts.values())
balance_ratio = min_samples / max_samples if max_samples > 0 else 0
print(f"4. Class balance ratio: {balance_ratio:.2f} {'✅ Good' if balance_ratio > 0.8 else '⚠️ Imbalanced'}")

# Check value ranges
print(f"\n5. Value Ranges:")
print(f"   Min value: {data_array.min():.4f}")
print(f"   Max value: {data_array.max():.4f}")
print(f"   Mean value: {data_array.mean():.4f}")
print(f"   Std deviation: {data_array.std():.4f}")

print("\n" + "="*60)
if not has_nan and not has_inf and actual_features == expected_features:
    print("✅ Data quality check PASSED!")
else:
    print("⚠️ Data quality issues detected. Review before proceeding.")
print("="*60)

---

## 🎯 Summary

You have successfully collected sign language gesture data!

### What's Next?
Proceed to **03_data_preprocessing.ipynb** to:
- Load and clean the collected data
- Normalize and augment the dataset
- Split into training, validation, and test sets
- Prepare data for model training

### Tips for Better Data Collection
- Collect data in different lighting conditions
- Vary hand positions and angles
- Include samples from different people (if possible)
- Ensure consistent gesture formation

---